# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298249791312                   -0.85    5.1
  2   -8.300181920845       -2.71       -1.26    1.9
  3   -8.300447711770       -3.58       -1.88    3.4
  4   -8.300462041564       -4.84       -2.68    1.0
  5   -8.300464420167       -5.62       -3.18    4.0
  6   -8.300464551028       -6.88       -3.30    7.5
  7   -8.300464587698       -7.44       -3.44    1.0
  8   -8.300464613985       -7.58       -3.57    1.0
  9   -8.300464636457       -7.65       -3.77    1.0
 10   -8.300464642437       -8.22       -3.96    1.0
 11   -8.300464643927       -8.83       -4.40    1.8


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.67382466490                   -0.70    5.5
  2   -16.67791960355       -2.39       -1.15    1.0
  3   -16.67919105373       -2.90       -1.88    4.5
  4   -16.67926640989       -4.12       -2.69    3.4
  5   -16.67928329169       -4.77       -3.03    6.0
  6   -16.67928620385       -5.54       -3.47    2.0
  7   -16.67928622081       -7.77       -3.99    1.1
  8   -16.67928622410       -8.48       -4.61    1.8


In [5]:
self_consistent_field(aluminium_setup(4); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.31653654821                   -0.56    6.1
  2   -33.33258251344       -1.79       -1.00    1.9
  3   -33.33411958083       -2.81       -1.73    4.2
  4   -33.33441970882       -3.52       -2.51    3.6
  5   -33.33693153056       -2.60       -2.54   15.4
  6   -33.33693801416       -5.19       -2.68    1.2
  7   -33.33694327496       -5.28       -3.28    1.6
  8   -33.33694368867       -6.38       -3.50    4.4
  9   -33.33694392228       -6.63       -4.32    1.8


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298549644104                   -0.85    5.1
  2   -8.300280800232       -2.76       -1.59    1.2
  3   -8.300453573368       -3.76       -2.59    5.1
  4   -8.300391934770   +   -4.21       -2.33    9.8
  5   -8.300464571028       -4.14       -3.81    2.5
  6   -8.300464609424       -7.42       -4.04    4.2


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32719771619                   -0.56    7.1
  2   -33.24357661581   +   -1.08       -1.23    1.5
  3   +34.36408805776   +    1.83       -0.15   10.4
  4   -33.32090402689        1.83       -1.69    7.5
  5   -32.97561121350   +   -0.46       -1.21    4.4
  6   -31.95455335489   +    0.01       -0.99    5.1
  7   -33.31652772519        0.13       -1.77    4.8
  8   -33.33006197984       -1.87       -2.05    2.1
  9   -33.33435373972       -2.37       -2.03    2.6
 10   -33.33687899519       -2.60       -2.91    2.0
 11   -33.33693831562       -4.23       -3.21    4.2
 12   -33.33694000541       -5.77       -3.33    2.1
 13   -33.33693387413   +   -5.21       -3.37    3.2
 14   -33.33694369845       -5.01       -4.03    2.4


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.